<a name="top"></a>
# AutoWISP
[![AutoWISP](https://github.com/kpenev/AutoWISP/blob/master/AutoWISP.png)](https://github.com/kpenev/AutoWISP)
[![language](https://img.shields.io/badge/Python-3776AB?logo=python&logoColor=fff)](https://www.python.org/about/)
[![OS](https://img.shields.io/badge/OS-linux%2C%20windows%2C%20macOS-0078D4)](https://github.com/kpenev/AutoWISP/tree/master/documentation)
[![CPU](https://img.shields.io/badge/CPU-x86%2C%20x64%2C%20ARM%2C%20ARM64-FF8C00)](https://github.com/kpenev/AutoWISP/tree/master/documentation)
[![GitHub release](https://img.shields.io/github/v/release/kpenev/AutoWISP)](#)
[![GitHub release date](https://img.shields.io/github/release-date/kpenev/AutoWISP)](#)
[![GitHub last commit](https://img.shields.io/github/last-commit/kpenev/AutoWISP)](#)
[![getting started](https://img.shields.io/badge/getting_started-guide-1D76DB)](https://github.com/kpenev/AutoWISP/tree/master/documentation)
[![Free](https://img.shields.io/badge/free_for_non_commercial_use-brightgreen)](#-license)

⭐ Star us on GitHub — it motivates us a lot!

## Table of Contents
- [About](#-about)
- [How to Install](#-how-to-install)
- [Documentation](#-documentation)
- [Feedback and Contributions](#-feedback-and-contributions)
- [License](#-license)
- [Contacts](#%EF%B8%8F-contacts)

## 🚀 About

**AutoWISP** is a Python package designed to allow users to automatically create and de-trend light curves from their astronomical data. It adheres to high standards of flexibility, reusability, and reliability, utilizing well-known software design patterns, including modular and hexagonal architectures. These patterns ensure the following benefits:

- **Modularity**: Different parts of the package can function independently, enhancing the package's modularity and allowing for easier maintenance and updates.
- **Testability**: Improved separation of concerns makes the code more testable.
- **Maintainability**: Clear structure and separation facilitate better management of the codebase.

## 📝 How to Install

To build the packages, follow these steps:

Use the package manager [pip](https://pip.pypa.io/en/stable/) to install AutoWISP.

    pip install autowisp

## 📚 Documentation

### Getting Started
Explore the [Getting Started Guide](https://github.com/kpenev/AutoWISP/tree/master/documentation).
In this guide, you will go step by step in the pipeline, producing the corresponding files needed for each step and ultimately creating light curves.

** See below for usage of each step, included is a configuration file to perform the tests.

To better understand the AutoWISP pipeline, we recommend visiting our [Documentation](https://github.com/kpenev/AutoWISP/tree/master/documentation) site.
There, you will find useful information about the individual steps, database, and/or browser-user-interface (currently under development).

## 🤝 Feedback and Contributions

We've made every effort to implement all the main aspects of AutoWISP in the best possible way. However, the development journey doesn't end here, and your input is crucial for our continuous integration and development.

> [!IMPORTANT]
> Whether you have feedback on features, have encountered any bugs, or have suggestions for enhancements, we're eager to hear from you. Your insights help us make the AutoWISP package more robust and user-friendly.

Please feel free to contribute by [submitting an issue](https://github.com/kpenev/AutoWISP/issues) or [joining the discussions](https://github.com/kpenev/AutoWISP/discussions). Each contribution helps us grow and improve.

We appreciate your support and look forward to making our pipeline even better with your help!

## 📃 License

This package is distributed under the MIT License. You can review the full license agreement at the following link: [MIT](https://choosealicense.com/licenses/mit/).

For non-commercial use, this product is available for free.

## 🗨️ Contacts

For more details about our usages, services, or any general information regarding the AutoWISP pipeline, feel free to reach out to us. We are here to provide support and answer any questions you may have. Below are the best ways to contact our team:

- **Email**: Send us your inquiries or support requests at [support_autowisp@gmail.com](mailto:support_autowisp@gmail.com).

We look forward to assisting you and ensuring your experience with AutoWISP is successful and enjoyable!

[Back to top](#top)


## Testing
The following will walk you through processing a very small collection of images to lightcurves.

### Create a temporary directory to work in (will be deleted at the end), enter it, and get its path

In [4]:
%mkdir autowisp_test
%cd autowisp_test

mkdir: autowisp_test: File exists
/Users/kpenev/projects/git/AutoWISP/autowisp/tests/test_data/autowisp_test


### Download the test data

The raw images processed in this example as well as the expected files that processing will produce can be downloaded from:
(https://zenodo.org/records/15786531/files/test_data.zip?download=1)

Download the zip file from zenodo and unpack it. This will produce several directories called: `CAL`, `DR`, `Gaia`, `LC`, `MASTERS`, and `RAW`. It should also produce several files, one of which is called `test.cfg`. 

Move `RAW` and `test.cfg` to the directory we just created. 

The remaining directories contain the files that processing should produce. You can compare what you get to those files.

### Perform the calibration for the zero frames

In [7]:
!wisp-calibrate --config-file test.cfg RAW/zero

This should produce a directory called `CAL` under which there will be another directory called `zero` which contains the calibrated bias frames.

### Create a master bias by stacking the calibrated zero frames

In [10]:
!wisp-stack-to-master -c test.cfg CAL/zero/

This should produce a directory called MASTERS containing a single FITS file, which is the master bias.

### Perform the calibration for the dark frames

Note that when calibrating the dark frames we wish to subtract the master bias generated above. In order to do this, we specify that the master that was generated above should be used as a master bias by adding `--master-bias "R:MASTERS/zero_R.fits.fz"` to our command line

In [13]:
!wisp-calibrate --config-file test.cfg RAW/dark/*.fits.fz --master-bias "R:MASTERS/zero_R.fits.fz"

There should now be another directory under `CAL`, called `dark` which contains the calibrated dark frames.

### Create a master dark by stacking the calibrated dark images

In [16]:
!wisp-stack-to-master -c test.cfg CAL/dark/

### Perform calibration for flat frames
Those should be corrected both with the master bias and the master dark generated in the previous two steps:

In [18]:
!wisp-calibrate --config-file test.cfg RAW/flat/*.fits.fz --master-bias "R:MASTERS/zero_R.fits.fz" --master-dark "R:MASTERS/dark_R.fits.fz"

### Build a master flat out of the individual calibrated flat frames

Note that creating a master flat uses a different command than creating master bias and dark!

In [20]:
!wisp-stack-to-master-flat -c test.cfg CAL/flat/

### Perform calibration for object frames
Now we use all three masters

In [22]:
!wisp-calibrate --config-file test.cfg RAW/object/*.fits.fz --master-bias "R:MASTERS/zero_R.fits.fz" --master-dark "R:MASTERS/dark_R.fits.fz" --master-flat "R:MASTERS/flat_R.fits.fz"

### Find stars in the calibrated object frames
This will create another directory called `DR` which contains what we call data reduction files. These files are where the pipeline saves  the results of processing the calibrated images. So fare, these files will only contain the stars that we find in the images.

In [24]:
!wisp-find-stars --config-file test.cfg CAL/object/

2025-07-03 10:40:15,077 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:40:15,080 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? ORDER BY hdf5_structure_versions.version DESC
 LIMIT ? OFFSET ?
2025-07-03 10:40:15,081 INFO sqlalchemy.engine.Engine [generated in 0.00036s] ('data_reduction', 1, 0)
2025-07-03 10:40:15,088 INFO sqlalchemy.engine.Engine SELECT hdf5_l

### Perform astrometry on DR files
This matches the extracted sources to a catalog of stars from the Gaia space mission and uses that information to figure out how to convert from coordinates on the sky to coordinates within each image. The conversion will also be stored in the DR files.

Note that this may take a while, since we use an online service to find an initial guess which is often over-subscribed.

In [26]:
!wisp-solve-astrometry --config-file test.cfg DR/

2025-07-03 10:40:19,060 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:40:19,062 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:40:19,062 INFO sqlalchemy.engine.Engine [generated in 0.00012s] ('data_reduction', 0)
2025-07-03 10:40:19,066 INFO sqlalchemy.engine.Engine SELECT hdf5_links.hdf5_structure_version

### Perform PSF/PRF fitting

In [28]:
!wisp-fit-star-shape --config test.cfg CAL/object/

2025-07-03 10:43:12,531 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:43:12,533 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:43:12,533 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ('data_reduction', 0)
2025-07-03 10:43:12,537 INFO sqlalchemy.engine.Engine SELECT hdf5_links.hdf5_structure_version

### Perform aperture photometry

In [30]:
!wisp-measure-aperture-photometry --config test.cfg CAL/object/

2025-07-03 10:43:25,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:43:25,853 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:43:25,853 INFO sqlalchemy.engine.Engine [generated in 0.00012s] ('data_reduction', 0)
2025-07-03 10:43:25,856 INFO sqlalchemy.engine.Engine SELECT hdf5_links.hdf5_structure_version

### Run fit source extracted PSF map

In [32]:
!wisp-fit-source-extracted-psf-map --config test.cfg DR/

2025-07-03 10:43:34,206 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:43:34,208 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:43:34,208 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ('data_reduction', 0)
2025-07-03 10:43:34,212 INFO sqlalchemy.engine.Engine SELECT hdf5_links.hdf5_structure_version

### Perform magnitude fitting

In [34]:
!wisp-fit-magnitudes --config test.cfg DR/

2025-07-03 10:43:37,719 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:43:37,721 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:43:37,721 INFO sqlalchemy.engine.Engine [generated in 0.00013s] ('data_reduction', 0)
2025-07-03 10:43:37,725 INFO sqlalchemy.engine.Engine SELECT hdf5_links.hdf5_structure_version

### Create light curves

In [36]:
!wisp-create-lightcurves --config test.cfg DR/

2025-07-03 10:44:21,419 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:44:21,421 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:44:21,421 INFO sqlalchemy.engine.Engine [generated in 0.00012s] ('data_reduction', 0)
2025-07-03 10:44:21,424 INFO sqlalchemy.engine.Engine SELECT hdf5_links.hdf5_structure_version

### Perform EPD on light curves

In [38]:
!wisp-epd --config test.cfg LC/

ERROR! Session/line number was not unique in database. History logging moved to new session 2
Parsing: 'sphotref = apphot.magfit.cfg.single_photref : aperture_index = 0'.
Parsing: "x = srcproj.columns : srcproj_column_name = 'x' & srcproj_version = 0".
Parsing: "y = srcproj.columns : srcproj_column_name = 'y' & srcproj_version = 0".
Parsing: 'bg = bg.value'.
Parsing: 'z = skypos.zenith_distance'.
Parsing: "S = srcextract.psf_map.eval: srcextract_psf_param = 's'".
2025-07-03 10:44:38,888 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:44:38,891 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5

### Generate EPD statistics file for light curves

In [40]:
!wisp-generate-epd-statistics --config test.cfg LC/

Parsing: 'sphotref = apphot.magfit.cfg.single_photref : aperture_index = 0'.
2025-07-03 10:44:43,338 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:44:43,340 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:44:43,340 INFO sqlalchemy.engine.Engine [generated in 0.00012s] ('data_reduction', 0)
2025-07-03 10:44:

### Perform TFA on light curves

In [42]:
!wisp-tfa --config test.cfg LC/

Parsing: 'sphotref = apphot.magfit.cfg.single_photref : aperture_index = 0'.
2025-07-03 10:44:47,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:44:47,225 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:44:47,225 INFO sqlalchemy.engine.Engine [generated in 0.00014s] ('data_reduction', 0)
2025-07-03 10:44:

### Generate TFA statistics file for light curves

In [44]:
!wisp-generate-tfa-statistics --config test.cfg LC/

Parsing: 'sphotref = apphot.magfit.cfg.single_photref : aperture_index = 0'.
2025-07-03 10:44:58,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 10:44:58,993 INFO sqlalchemy.engine.Engine SELECT hdf5_structure_versions.hdf5_product_id AS hdf5_structure_versions_hdf5_product_id, hdf5_structure_versions.version AS hdf5_structure_versions_version, hdf5_structure_versions.id AS hdf5_structure_versions_id, hdf5_structure_versions.timestamp AS hdf5_structure_versions_timestamp, hdf5_products.pipeline_key AS hdf5_products_pipeline_key, hdf5_products.description AS hdf5_products_description, hdf5_products.id AS hdf5_products_id, hdf5_products.timestamp AS hdf5_products_timestamp 
FROM hdf5_products JOIN hdf5_structure_versions ON hdf5_products.id = hdf5_structure_versions.hdf5_product_id 
WHERE hdf5_products.pipeline_key = ? AND hdf5_structure_versions.version = ?
2025-07-03 10:44:58,993 INFO sqlalchemy.engine.Engine [generated in 0.00012s] ('data_reduction', 0)
2025-07-03 10:44:

When you are done reviewing your results, remember to delete the directory you created at the beginning